In [1]:
# libraries
import random
import numpy as np
import pickle
import json
from flask import Flask, render_template, request
# from flask_ngrok import run_with_ngrok
import nltk
from tensorflow.keras.models import load_model
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [2]:
# chat initialization
model_lstm=load_model("model_lstm.h5")
model = load_model("chatbot_model.h5")
intents = json.loads(open("intents.json").read())
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))

In [3]:
app = Flask(__name__)
# run_with_ngrok(app) 

@app.route("/")
def home():
    return render_template("index.html")

In [4]:
# # @app.route("/get", methods=["POST"])
# @app.route("/get")
# def chatbot_response():
#     msg = request.args.get('msg')
#     if (msg.startswith('my name is') or msg.startswith('I\'m') or msg.startswith('I am')):
#         name = msg[11:]
#         ints = predict_class(msg, model)
#         res1 = getResponse(ints, intents)
#         res =res1.replace("{n}",name)
#     elif (msg.startswith('hi my name is') or msg.startswith('hi I\'m')):
#         name = msg[14:]
#         ints = predict_class(msg, model)
#         res1 = getResponse(ints, intents)
#         res =res1.replace("{n}",name)
# #         print('elif 1 '+res)
#     elif msg.startswith('description:'):
#         msg = msg.partition('description:')[2]
#         print(msg)
#         res = predict_acc_level(msg,model_lstm)    
# #         print('elif 2 '+res)
#     else:
#         ints = predict_class(msg, model)
#         res = getResponse(ints, intents)
# #         print('else 1 '+res)
#     return res

In [5]:
# @app.route("/get", methods=["POST"])
@app.route("/get")
def chatbot_response():
    msg = request.args.get('msg')
    if (msg.startswith('my name is') or msg.startswith('I\'m') or msg.startswith('I am'.lower())):
        name = msg.split()[-1]
        ints = predict_class(msg, model)
        res1 = getResponse(ints, intents)
        res =res1.replace("{n}",name)
    elif (msg.startswith('hi my name is') or msg.startswith('hi I\'m')):
        name = msg.split()[-1]
        ints = predict_class(msg, model)
        res1 = getResponse(ints, intents)
        res =res1.replace("{n}",name)
#         print('elif 1 '+res)
    elif msg.startswith('description:'):
        msg = msg.partition('description:')[2]
        print(msg)
        res = predict_acc_level(msg,model_lstm)    
#         print('elif 2 '+res)
    else:
        ints = predict_class(msg, model)
        res = getResponse(ints, intents)
#         print('else 1 '+res)
    return res

In [6]:
# chat functionalities
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# def get_bot_response():
#     userText = request.args.get('msg')
#     return str('Accident level unknown')

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)


def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]["intent"]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
#         inp = input()
#         if inp.lower() == "quit":
#             break
    return result


In [7]:
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
# add appropriate words that will be ignored in the analysis
ADDITIONAL_STOPWORDS = ['covfefe']
# nlp libraries
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import string
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from nltk import tokenize,stem

# Keras pre-processing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Karthick\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Karthick\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Karthick\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
def text_cleaning(text):
   # Initialize the object for Lemmatizer class
    lemmatizer = nltk.stem.WordNetLemmatizer()

    # Set the stopwords to English
    stopwords = nltk.corpus.stopwords.words('english')

    # Normalize the text in order deal with accented words and unicodes
    text = (unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore').lower())

    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    #word tokenization
    words = nltk.word_tokenize(text.translate(remove_punct_dict))

    # Consider the words which are not in stopwords of english and lemmatize them
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lems = [lemmatizer.lemmatize(i) for i in words if i not in stopwords]

    return lems

In [9]:
def predict_acc_level(msg,model):
    data = " ".join(text_cleaning(msg))
    tokenizer = Tokenizer(num_words=5000)
    #vectorizing the tweet by the pre-fitted tokenizer instance
    data = tokenizer.texts_to_sequences(data)
    #padding the tweet to have exactly the same shape as `embedding_2` input
    data = pad_sequences(data, maxlen=100, dtype='int32', value=0)
    # print(data)
    ypred = model.predict(data,batch_size=1,verbose = 2)[0]
    print(ypred)
    class_label=np.argmax(ypred)
    if class_label == 0:
        result ='AccidentLevel I'
    elif class_label == 1:
        result ='AccidentLevel II'
    else:
        result ='AccidentLevel III'
    print(result)
    return result

In [10]:
predict_acc_level('approximately 1145 circumstance mechanic anthony group leader',model_lstm)

61/61 - 1s - 1s/epoch - 24ms/step
[0.809407   0.08644677 0.10414631]
AccidentLevel I


'AccidentLevel I'

In [11]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Dec/2021 18:35:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2021 18:35:12] "GET /style.css HTTP/1.1" 404 -
127.0.0.1 - - [29/Dec/2021 18:35:13] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [29/Dec/2021 18:35:15] "GET /get?msg=hi HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2021 18:35:20] "GET /get?msg=my%20name%20is%20kar HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2021 18:35:30] "GET /get?msg=yes HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2021 18:35:44] "GET /get?msg=no HTTP/1.1" 200 -
